## Erstellen eine Wörterbuchs mit speziellen Begriffen der Stabi

In [1]:
!abiword --to=txt ../stabi-dict/rak_musik_2003.pdf

In [ ]:
!aspell -l de_DE dump master | aspell -l de expand | tr ' ' '\n' > de_DE.dict

In [113]:
from collections import Counter
import unicodedata, re

# source: https://stackoverflow.com/questions/92438/stripping-non-printable-characters-from-a-string-in-python
all_chars = (unichr(i) for i in xrange(0x110000))
control_chars = ''.join(c for c in all_chars if unicodedata.category(c) == 'Cc')
# or equivalently and much more efficiently
control_chars = ''.join(map(unichr, range(0,32) + range(127,160)))

control_char_re = re.compile('[%s\d]' % re.escape(control_chars))

not_words = re.compile(u'[\(\),:<>\*\-\=\[\]\?\!\.;„“]')
digits = re.compile('[\d]*')

def remove_control_chars(s):
    return control_char_re.sub('', s)

text = open('../stabi-dict/rak_musik_2003.txt').read().decode('utf-8')
words = Counter(re.split('[\s\n\/]', text)).keys()

words = [control_char_re.sub('', w) for w in words]
words = [not_words.sub('', w) for w in words]
words = [digits.sub('', w) for w in words]
words = [w for w in words if not len(w) < 2]
words = set(words)
words = words.difference(set(re.split('\n', open('../stabi-dict/de_DE.dict').read().decode('utf-8'))))
words = list(words)
words.sort()

with open('../stabi-dict/de_stabi.dict', 'wb') as f:
    f.write((u'\n'.join(words).encode('utf-8')))